In [2]:
import gdrive.GDriveHandler as GDH
import pandas as pd
import numpy as np
import pytorch_lightning.core as L
from torch.utils.data import DataLoader,Dataset
import GDriveDataloading as GDTL
import matplotlib.pyplot as plt
import os
import sys
from PIL import Image,ImageShow
import numpy as np
import torch
import seaborn as sb
import matplotlib.pyplot as plt
import yaml
from omegaconf import OmegaConf
import torchvision.transforms as transforms
import time
import main
import taming.modules.losses.vqperceptual
from taming.models.cond_transformer import Net2NetTransformer
from taming.models.vqgan import  LAPVQ



ID_DF_Path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers2\scripts\dataloader\gdrive\FFHQimages.csv"
config_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers2\configs\gdrive_FFHQ.yaml"
scopes = ["https://www.googleapis.com/auth/drive.readonly"]
creds_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers2\scripts\dataloader\gdrive\deep-learning-2023-405822-135193813109.json"
access_token_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers2\scripts\dataloader\test.json"

GDDataloader = GDTL.ImagesDatamodule(ID_DF_Path,scopes,creds_path,num_workers=0,batch_size=3)
prop_dict = dict({"val":0.1,"test":0.1,"train":0.8})
GDDataloader.setup(prop_dict)

train_dataloader = GDDataloader.train_dataloader()
start = time.time()
iter_train = iter(train_dataloader)

#Prepare CelebAHQ configurations
config_path = r"C:\Users\DripTooHard\PycharmProjects\taming-transformers2\configs\faceshq_vqgan.yaml"
celebAHQ_config = OmegaConf.load(config_path)
print(yaml.dump(OmegaConf.to_container(celebAHQ_config)))

#Init model with the chosen architecture and configurations
model = LAPVQ(**celebAHQ_config.model.params)




data:
  params:
    batch_size: 3
    num_workers: 8
    train:
      params:
        crop_size: 256
        size: 256
      target: taming.data.faceshq.FacesHQTrain
    validation:
      params:
        crop_size: 256
        size: 256
      target: taming.data.faceshq.FacesHQValidation
  target: main.DataModuleFromConfig
model:
  base_learning_rate: 4.5e-06
  params:
    ddconfig:
      attn_resolutions:
      - 16
      ch: 128
      ch_mult:
      - 1
      - 1
      - 2
      - 2
      - 4
      double_z: false
      dropout: 0.0
      in_channels: 3
      num_res_blocks: 2
      out_ch: 3
      resolution: 256
      z_channels: 256
    embed_dim: 256
    lossconfig:
      params:
        codebook_weight: 1.0
        disc_conditional: false
        disc_in_channels: 3
        disc_start: 30001
        disc_weight: 0.8
      target: taming.modules.losses.vqperceptual.VQLPIPSWithDiscriminator
    n_embed: 1024
  target: taming.models.vqgan.LAPVQ

Working with z of shape (1, 256, 16,

C:\Users\DripTooHard\PycharmProjects\taming-transformers2\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DripTooHard\PycharmProjects\taming-transformers2\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips\vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


In [4]:
model.set_epsilon(10)